In [ ]:
#Import required packages
import cv2 as cv
import numpy as np
import pandas as pd
import cv2 as cv
import numpy as np
import pandas as pd
from Helper_functions import interactive_session, load_dict, make_tile
import umap

#Insert filepath for local files  FOR THIBAUT
basepath = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Data\BoneMarrow_smallerfile2"
readpath = basepath + "\\Raw_Images"
outpath = basepath + "\\Outputs"
image_dim=64 #Dim of the final images
nuclear_channel="Ch7"
cellmask_channel="Ch1_mask"

df=pd.read_csv(outpath+"\\cell_info.csv")
cell_names=df["Cell_ID"].to_numpy()

Channels=['Ch1']  #Channel to be fed to the NN

image_dict=load_dict(outpath,cell_names,image_dim)
images_with_index = []
for image_i in image_dict:
    image=cv.merge([image_dict[image_i][i] for i in Channels])
    images_with_index.append((int(image_i),image))
    
images=np.array([image[1] for image in images_with_index])
names=np.array([image[0] for image in images_with_index])
labels=df['Cell_Type'].to_numpy()
assert sum(names!=df['Cell_ID'].to_numpy()) ==0  #Check that the order has been preserved
DNA_pos=df["DNA_pos"].to_numpy()
Touches_Boundary=df["Touches_boundary"].to_numpy()
labels=df['Cell_Type'].to_numpy()

latent_df=pd.read_csv(basepath+"\\Results\\"+"Barlow_latents433.csv")
namesdf=latent_df["Cell_ID"].to_numpy()
display_images=[image_dict[int(i)]['Ch1'] for i in namesdf]
barlow=latent_df.drop("Cell_ID",axis=1).to_numpy()
labelsdf=[labels[np.where(names==int(i))] for i in namesdf]

colors=[]
for label in labelsdf:
    if int(label)==0:
        colors.append('red')
    elif int(label)==2:
        colors.append('blue')


In [ ]:
fit = umap.UMAP(n_components=2)
u = fit.fit_transform(barlow)
interactive_session(u,display_images,colors,namesdf)


In [ ]:
import dash
print(dash.__version__)

In [ ]:
fit = umap.UMAP(n_components=3)
u = fit.fit_transform(barlow)
interactive_session(u,display_images,colors,namesdf)


In [ ]:
fit = umap.UMAP(n_components=1)
u = fit.fit_transform(barlow)
make_tile(display_images=display_images,u=u,n=100,savepath=False)

In [ ]:
df=pd.read_csv(basepath+"//Results//Resnet18_70.41//Loss.csv")

In [ ]:
a=list(df["train_acc"].to_numpy())
b=list(df["val_acc"].to_numpy())

In [ ]:
a=a[::-1]
a.append(50)
a=a[::-1]

In [ ]:
b=b[::-1]
b.append(50)
b=b[::-1]

In [ ]:
import matplotlib.pyplot as plt
plt.plot(a,label="Train accuracy")
plt.plot(b,label="Validation accuracy")
plt.legend()

In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
from Helper_functions import interactive_session, load_dict, make_tile, adjust_contrast
import umap
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [ ]:
#Import required packages
import cv2 as cv
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2 as cv
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import pandas as pd
from AI_functions import resnet18, BarlowTwins, ImageCollateFunction,BarlowTwinsLoss,CellDataset,data_generator


In [ ]:
#This is for HEROKU
DATASETS=np.array(["BoneMarrow_sample1","Retina_1_2","Colon","Choroid"])

basepath = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Results"
outpath = basepath + "\\Results"
df_path="\Barlow_latents104.csv"


df=pd.read_csv(basepath+df_path)
cell_names=df["Cell_ID"].to_numpy()
Labels=df["Dataset"].to_numpy()

latents=df.iloc[:,1:].drop(["Cell_ID","Dataset","U1","U0"],axis=1).to_numpy()
print(latents.shape)


In [ ]:
display_images=np.zeros((len(cell_names),64,64))
for DATASET in np.unique(Labels):
    print(DATASET)
    image_path = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Data\\" +DATASET +"\\Outputs"
    cell_info=pd.read_csv(image_path+"\\cell_info.csv")
    names=cell_info["Cell_ID"].to_numpy()
    cell_names_subset=cell_names[np.where(Labels==DATASET)]
    image_dict=load_dict(image_path,names,64)
    display_images[np.where(Labels==DATASET)]=[image_dict[int(i)]['Ch1'] for i in cell_names_subset]  #This line is just beautiful




In [ ]:
COLOR_DICT={"BoneMarrow_sample1":"red","Retina_1_2":"limegreen","Colon":"magenta","Choroid":"blue"}
inv_map = {v: k for k, v in COLOR_DICT.items()}
colors=[COLOR_DICT[i] for i in Labels]

In [ ]:
fit = TSNE(n_components=2)
u = fit.fit_transform(latents)

df.to_csv(basepath+df_path)

#u0=df["U0"]
#u1=df["U1"]


In [ ]:
df["U0"]=u[:,0]
df["U1"]=u[:,1]
df.to_csv(basepath+df_path)

In [ ]:
import matplotlib.patches as mpatches

fig=plt.figure(figsize=(15,10))

patches=[mpatches.Patch(color=color, label=tissue) for tissue,color in COLOR_DICT.items()]
plt.legend(handles=patches)
plt.scatter(u[:,0],u[:,1],c=colors,s=1)

In [ ]:
import matplotlib.patches as mpatches

fig=plt.figure(figsize=(15,10))

patches=[mpatches.Patch(color=color, label=tissue) for tissue,color in COLOR_DICT.items()]
plt.legend(handles=patches)
plt.scatter(u[:,0],u[:,1],c=colors,s=1)

In [ ]:
interactive_session(u,display_images,colors,cell_names)

In [ ]:
display_images-=np.min(display_images)
display_images/=np.max(display_images)
display_images*=255
adjusted_contrast=[255-cv.convertScaleAbs(i, alpha=4, beta=-140) for i in display_images]

In [ ]:
np.save(r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\HEROKU_Atlas\DATA1\\"+"Images"+"_Ch1.npy",adjusted_contrast)